# 🔍 Building an AI-Powered Web Search Agent with OpenAI and Tavily 🚀

Hey there! Welcome to this exciting guide where we'll create something awesome - a smart search agent that combines the power of OpenAI's language models with Tavily's search capabilities! 🌟 

## 🎯 What We'll Build

We're going to create a super cool search agent that can:
1. 🌐 Search the web in real-time for accurate information
2. 🧠 Use OpenAI's powerful GPT models to understand and process search results
3. ⚡ Provide contextual and up-to-date responses to queries

## ✅ Prerequisites

Before we jump in, make sure you have these things ready:
- 🔑 An OpenAI API key
- 🎯 A Tavily API key (get one at tavily.com)

## 🎮 Part 1: Setting Up Our Environment

First things first - let's get our tools ready! We'll need to install the Tavily Python package to interact with their search API:


In [13]:
%pip install tavily-python

Note: you may need to restart the kernel to use updated packages.



Now, let's set up our API keys (remember to keep these safe and never share them!):


In [14]:
TAVILY_API = "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"


## 🛠️ Part 2: Building Our Search Tools

Let's create the foundation of our search agent! We'll define a set of tools that our AI can use to search the web:

In [15]:
import json
from openai import OpenAI
from tavily import TavilyClient
import os
import sys
import os

# Add the parent directory to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from agent.prompts import get_system_prompt

In [16]:
# Initialize Tavily
tavily = TavilyClient(api_key=TAVILY_API)

# Search query
query = "Whats the current weather in Tokyo"
tavily.search(query, search_depth="basic")

{'query': 'Whats the current weather in Tokyo',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Weather in Tokyo',
   'url': 'https://www.weatherapi.com/',
   'content': "{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.6895, 'lon': 139.6917, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1739958125, 'localtime': '2025-02-19 18:42'}, 'current': {'last_updated_epoch': 1739957400, 'last_updated': '2025-02-19 18:30', 'temp_c': 6.1, 'temp_f': 43.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 21.7, 'wind_kph': 34.9, 'wind_degree': 345, 'wind_dir': 'NNW', 'pressure_mb': 1014.0, 'pressure_in': 29.94, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 33, 'cloud': 25, 'feelslike_c': 1.1, 'feelslike_f': 34.0, 'windchill_c': -0.4, 'windchill_f': 31.4, 'heatindex_c': 5.0, 'heatindex_f': 41.0, 'dewpoint_c': -13.6, 'dewpoint_f': 7.4, 'vis_km': 1

In [17]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "Search the web for information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"}
                },
                "required": ["query"]
            },
        },
    },
]

In [18]:
def search_web(query):
    """Searches for both text information and images about a destination."""

    tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

    # Search for general country info
    text_results = tavily.search(query, search_depth="basic")["results"]
    text_info = "\n".join(result["content"] for result in text_results)

    # Search for images
    image_results = tavily.search(query, search_type="image")["results"]
    image_urls = [result["url"] for result in image_results if result["url"].endswith(("jpg", "png", "jpeg", "webp"))]

    return {"text": text_info, "images": image_urls}

In [19]:
search_web("A trip to Tokyo for 3 days")

{'text': "In just 3 days in Tokyo, you will see the most iconic neighborhoods such as Shibuya, Shinjuku, and Asakusa, visit cultural sites like Senso-ji Temple and the Meiji Shrine, and experience the city's best food scene. So, settle down with a nice, cup of hot tea and get ready to explore the best of Tokyo during your unforgettable 3-day trip.\n3 Days in Tokyo Itinerary Day 1 Morning: teamLab Borderless + Coffee teamLab Borderless. Your first stop in Tokyo is the mind-bending digital art museum, teamLab Borderless.This immersive experience is one of the most popular attractions in Tokyo, and for good reason!. Imagine walking through a space where the boundaries between art, technology, and imagination completely dissolve—rooms\nWith that said, 3 days in Tokyo is not enough to see Tokyo. But, it is enough to see certain icons - including wards like Shibuya, Shinjuku, and Roppongi. If you have the time, I recommend spending 5 days in Tokyo - during which you'll be able to have a full


## 🎓 Part 3: Creating Our AI Agent

Now comes the exciting part! Let's create our AI agent that can understand questions and use our search tools to find answers:


In [20]:
messages = [
    {"role": "system", "content": get_system_prompt()},
    {"role": "user", "content": "Plan me a trip of 5 days to Tokyo, Japan"}
]

In [21]:
def invoke_model(messages):
    # Initialize the OpenAI client
    client = OpenAI()

    # Make a ChatGPT API call with tool calling
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message.content

In [ ]:
# Initialize the OpenAI client
client = OpenAI()

# Make a ChatGPT API call with tool calling
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    tools=TOOLS,
    messages=messages
)

response = completion.choices[0].message

# Parse the response to get the tool call arguments
if response.tool_calls:
    # Process each tool call
    for tool_call in response.tool_calls:
        # Get the tool call arguments
        tool_call_arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "search_web":
            print(f"Searching the web: {tool_call_arguments["query"]}")
            search_results = search_web(tool_call_arguments["query"])
            messages.append({"role": "assistant", "content": f"{tool_call_arguments["query"]}: {search_results}"})

            
    print({
            "type": "info",
            "text": search_results["text"],
            "images": search_results["images"][:3]  # Limit to 3 images
        })

else:
    # If there are no tool calls, return the response content
    print(response.content)

Searching the web: natural attractions in Tokyo Japan with images
Searching the web: typical food in Tokyo Japan with local restaurants
Searching the web: activities to do in Tokyo Japan
{'type': 'info', 'text': "17 Best Things to do in Tokyo, Japan (2025 Travel Itinerary) Discover the sprawling metropolis of Tokyo, the capital city of Japan — home to weird and wonderful sights, neon flashing lights, expansive gardens, tavern-filled alleys, and sensory food markets. No trip to Tokyo would be complete without a visit to Mount Fuji, and the good news is that it’s easily accessible on a day trip! One of the best things to do in Tokyo, the Hie shrine is definitely worth a stop on your city trip. Tokyo is an excellent jumping-off point for day trips in Japan, primarily because of the superfast bullet trains that can take you out of the city in a matter of minutes. Costs of Traveling in Tokyo Best Time to Visit Tokyo\nThis is THE guide to our 23 fun things to do in Tokyo, Japan; from watchin